## Gauss Method


This is Gauss method elimination for solving systems $AX=b$ by solving an easier equivalent
system $TX = b^{'}$ where T is a triangular matrix.
for example we have the system 

$\begin{cases} -2x + y + -z = 8 \;\; (L1) \\ -3x – y + 2z = -11 \;\;(L2) \\ -2x + y + -2z = -3 \;\;(L3)\end{cases}$   which can be rewritten : $\begin{pmatrix} 
-2 & 1 & -1 \\
-3 & -1 & 2 \\
-2 & 1 & -2
\end{pmatrix} \begin{pmatrix} 
x  \\
y   \\
z
\end{pmatrix}= \begin{pmatrix} 
8  \\
-11   \\
3
\end{pmatrix}$

When you have linear system and you do linear operations between lines $L_i$ or columns $C_i$ the system obtained stay equivalent so this algorithm do the necessary linear operation to move from $A$ to $T$
the algorithm is expressed as follow at each $kth$ matrix iteration : <br/>
for $A \in M_n(R)$ $\forall i \in \; [k,n] L_i^{(k+1)} = L_i^{(k)} - m_i^{(k)} L_k$ where $m_i^{(k)} = \frac{a_{ik}}{a_{kk}}$  <br> Note that $a_{kk}$ is called the pivot  and $k \in [2, n]$

for more detail check wikipedia [link](https://en.wikipedia.org/wiki/Gaussian_elimination)

In [1]:
# Let code the Gauss method
import numpy as np
A = np.array([[-2,1,-1],[-3,-1,2],[-2,1,2]])
b = np.array([8, -11, -3]).reshape(3,1)
print("Matrix A\n",A)
print("\nVector b\n",b)

Matrix A
 [[-2  1 -1]
 [-3 -1  2]
 [-2  1  2]]

Vector b
 [[  8]
 [-11]
 [ -3]]


In [2]:
# we stack the matrix as we can do the lines operations in one time
stackA = np.hstack((A,b))
stackA

array([[ -2,   1,  -1,   8],
       [ -3,  -1,   2, -11],
       [ -2,   1,   2,  -3]])

In [3]:
def gauss(stackA):
    #we convert to np.float 64 because we do divisions that convert thing in float, so to avoid int cast we make it float
    T = np.array(stackA).astype(np.float64)
    for k in range(0, T.shape[0] - 1):
        p = 0
        for i in range(k + 1,T.shape[0]):
            m = T[i,k]/T[k,k]
            el = T[i] - (T[k] * m)
            T[i] +=  - (T[k] * m)
    return T
        

In [4]:
T = gauss(stackA)
#Triangulat uppper matrix
T

array([[ -2. ,   1. ,  -1. ,   8. ],
       [  0. ,  -2.5,   3.5, -23. ],
       [  0. ,   0. ,   3. , -11. ]])

In [6]:
def solve(T, b0):
    n = len(b0)
    #we solve the last esuation (line) in the matrix
    X = [b0[-1]/T[n - 1, n - 1]]
    #we go upper to solve the equations above using the last variable solved
    for k in range(n - 2, -1,-1):
        a = np.sum([X[(n - 1) - i] * T[k , i] for i in range(n - 1, k, -1)])
        term = (1/(T[k,k])) * (b0[k] - a)
        X.append(term)
    # we reverse W because, we solve it from down to up
    return np.array(X[::-1]).reshape(n,1)

In [7]:
X = solve(T[:,:3], T[:,3:])
a = (A @ X).astype(np.float)
b = b.astype(np.float)


In [8]:
# We use np.allclose because we manipulate float and we have some small imprecisions that make the test equality fail
np.allclose(a,b)

True

So now you are happy i guess, you know how to solve a system with a complexity of $O_3$, congratulations, 
but you're not totally up to it, what if the pivot $a_{kk} = 0$ how you handle it ? 

Do'nt worry you simply permut the lines (or columns) in order to not have $a_{kk} = 0$,more than that
is better to permut each time to have the bigest (in absolute value) $a_{kk}$ possible to have more stable divisions, so we are going to modify slightly
the preceding algorithm

for more information about permutation matrix see wikipedia [link](https://en.wikipedia.org/wiki/Permutation_matrix)


In [9]:
#This a permutation Matrix P that when multiplied by M, we permut for M the ith and jth line
def permutation_matrix(rang, i, j):
    P = np.identity(rang).astype(np.float64)
    tmp = P[j].copy()
    P[j] = P[i].copy()
    P[i] = tmp
    return P

In [10]:
M = np.array([[i + j for i in range(4) ] for j in range(4)])
M

array([[0, 1, 2, 3],
       [1, 2, 3, 4],
       [2, 3, 4, 5],
       [3, 4, 5, 6]])

In [11]:
P = permutation_matrix(4,0,1)
P

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [12]:
P @ M

array([[1., 2., 3., 4.],
       [0., 1., 2., 3.],
       [2., 3., 4., 5.],
       [3., 4., 5., 6.]])

In [13]:
def gauss_permut(stackA):
    #we convert to np.float 64 because we do divisions that convert thing in float, so to avoid int cast we make it float
    T = np.array(stackA).astype(np.float64)
    for k in range(0, T.shape[0] - 1):
        # We add those to lines that look for the max pivot each time and do the permutations
        j = np.argmax(abs(T[:,k]))
        T = np.array(permutation_matrix(T.shape[0],k,j) @ T)
        for i in range(k + 1,T.shape[0]):
            m = T[i,k]/T[k,k]
            el = T[i] - (T[k] * m)
            T[i] +=  - (T[k] * m)
    return T
        

In [14]:
#np.set_printoptions(formatter={'float_kind':float_formatter})

In [15]:
T = gauss_permut(stackA)
#Triangular uppper matrix different from the previous one
#We use float_formatter just to reformat for obtaining nicer display
float_formatter = lambda x: "%.2f" % x
np.array([[float_formatter(el) for el in line] for line in T])

array([['-3.00', '-1.00', '2.00', '-11.00'],
       ['0.00', '1.67', '-2.33', '15.33'],
       ['0.00', '0.00', '3.00', '-11.00']], dtype='<U6')

In [16]:
X = solve(T[:,:3], T[:,3:])
a = (A @ X).astype(np.float)
b = b.astype(np.float)


In [17]:
# as expected it should be true
np.allclose(a,b)

True

### Thank you ! 
I whish i have puted some clarity to the gauss elimination method for you, Thank you ! 